In [1]:
import random
from typing import List, Tuple


P = 2**127 - 1
Q = P - 1              


G = 3
H = 5

def pedersen_commit_scalar(m: int, r: int) -> int:

    m = int(m)
    r = int(r)
    return (pow(G, m, P) * pow(H, r, P)) % P

def pedersen_commit_vector(x: List[int], s: List[int]) -> List[int]:

    assert len(x) == len(s)
    return [pedersen_commit_scalar(x[i], s[i]) for i in range(len(x))]


In [2]:
def sample_instance(ell: int, B: int) -> Tuple[List[int], List[int], List[int]]:

    x = [random.randint(0, B) for _ in range(ell)]
    s = [random.randint(0, Q - 1) for _ in range(ell)]
    com_x_s = pedersen_commit_vector(x, s)
    return x, s, com_x_s

# 示例：维度 ell=4，界 B=10
ELL = 4
B = 10

x, s, com_x_s = sample_instance(ELL, B)

print("x =", x)
print("s =", s)
print("com(x,s) =", com_x_s)


x = [6, 3, 3, 5]
s = [44026465757881662910533140528340926188, 99567317077906853801615067455830061475, 169249068367155961434758248004994605126, 80775015929970521398576953196327970272]
com(x,s) = [150971967899089875101073078550403439779, 153496797595909143816343302190666952489, 107445841748706852906272074901455259066, 65212775189028049760483316291772357275]


In [3]:
def prover_first_message(ell: int, B: int):

    mu = [random.randint(0, B) for _ in range(ell)]
    rho = [random.randint(0, Q - 1) for _ in range(ell)]
    com_mu_rho = pedersen_commit_vector(mu, rho)
    return mu, rho, com_mu_rho

mu, rho, com_mu_rho = prover_first_message(ELL, B)

print("μ  =", mu)
print("ρ  =", rho)
print("com(μ,ρ) =", com_mu_rho)


μ  = [0, 2, 6, 2]
ρ  = [29742375609800961437947842205799289059, 79948097603236554015242149879294198560, 138517350185669883849627324743356768067, 96551080934889169923265614917287611622]
com(μ,ρ) = [160135783950706427496525210614892510010, 54769327751115368877237019972354344912, 23416227934574815491832246105276560840, 24792989903755968630084626560429801437]


In [4]:
def verifier_challenge(max_c: int = 2**16) -> int:

    return random.randint(0, max_c)

c = verifier_challenge()
print("challenge c =", c)


challenge c = 45645


In [5]:
def prover_response(x: List[int], s: List[int],
                    mu: List[int], rho: List[int], c: int):

    assert len(x) == len(mu) == len(s) == len(rho)
    Rx = [c * x[i] + mu[i] for i in range(len(x))]
    Rs = [c * s[i] + rho[i] for i in range(len(s))]
    return Rx, Rs

Rx, Rs = prover_response(x, s, mu, rho, c)
print("R_x =", Rx)
print("R_s =", Rs)


R_x = [273870, 136937, 136941, 228227]
R_s = [2009617771894118304512723147258327375140319, 4544830136118661578328734996171242450224935, 7725512242969019529573389857512722107744337, 3687072153204439338407968294261307490677062]


In [6]:
def linfty_norm(vec: List[int]) -> int:
    return max(abs(v) for v in vec) if vec else 0

def verifier_check(com_x_s: List[int],
                   com_mu_rho: List[int],
                   Rx: List[int], Rs: List[int],
                   c: int, B: int) -> bool:

    ell = len(com_x_s)
    assert ell == len(com_mu_rho) == len(Rx) == len(Rs)


    for j in range(ell):
        left = (pow(com_x_s[j], c, P) * com_mu_rho[j]) % P
        right = (pow(G, Rx[j], P) * pow(H, Rs[j], P)) % P
        if left != right:
            print(f"[eq fail] coordinate {j}: left != right")
            return False

    if linfty_norm(Rx) > (c + 1) * B:
        print("[bound fail] ||R_x||∞ >", (c + 1) * B)
        return False

    return True

accepted = verifier_check(com_x_s, com_mu_rho, Rx, Rs, c, B)
print("Verifier accepts?" , accepted)


Verifier accepts? True
